In [4]:
raw_data = [
    {"id": "101", "prompt": "How do I fix a flat tire?", "response": "<div>Use a jack...</div>", "score": 10},
    {"id": "102", "prompt": "How do I fix a flat tire?", "response": "Use a jack and a lug wrench...", "score": 15}, # Duplicate prompt, better response
    {"id": "103", "prompt": "Help me", "response": "Details?", "score": -1}, # Low quality
    {"id": "101", "prompt": "How do I fix a flat tire?", "response": "<div>Use a jack...</div>", "score": 10}, # Exact duplicate ID and content
    {"id": "104", "prompt": "Best python resources?", "response": None, "score": 5}, # Missing value
]

In [5]:
import pandas as pd
'''
Your Task: Write a Python function clean_and_curate(data) that performs the following curation steps:

Deduplication: If there are multiple entries with the exact same prompt, keep only the one with the highest score.

Cleaning: Remove any HTML tags (like <div>) from the response.

Filtering: discard any entries where the cleaned response is missing (None), empty, or shorter than 20 characters.
'''
import pandas as pd 
raw_data = pd.DataFrame(raw_data)

In [6]:
raw_data

,id,prompt,response,score
0,101,How do I fix a flat tire?,<div>Use a jack...</div>,10
1,102,How do I fix a flat tire?,Use a jack and a lug wrench...,15
2,103,Help me,Details?,-1
3,101,How do I fix a flat tire?,<div>Use a jack...</div>,10
4,104,Best python resources?,None,5


In [7]:
# three syntac 
# df.sort_values(by="score")
#df.groupby(by=["city", "year"])
#df.drop_duplicates(subset="id") 
def clean_and_curate(df):
    df = df.sort_values(by="score", ascending=True)
    df = df.drop_duplicates(subset= "prompt", keep ="last")
    df["response"] = df["response"].str.replace(r'<[^>]+>', '', regex=True)
    #df["response"] = df["response"].str.replace("<div>", "")
    # filtering discard any entries where the cleaned response is missing (None), empty, or shorter than 20 characters.
    df = df[df["response"].notna() & (df["response"].str.len() > 20)]
    return df 

In [2]:
# read a json strema 

import json 
def get_error_logs(filename):
    with open(filename,'r') as f:
        for line in f: 
            try: 
                json_line = json.loads(line)
                if json_line["status"] == "error":
                    yield json_line # this has to be inside a functoin only 
            except:
                continue
#log_stream = get_error_logs("logs.jsonl")

In [8]:
df = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2],
    'timestamp': pd.to_datetime([
        '10:00', '10:25', '11:30', # User 1: 25min gap (same), 65min gap (NEW)
        '09:00', '09:15'           # User 2: 15min gap (same)
    ])
})
df= df.sort_values(by = ["user_id", "timestamp"]) 
df["time_diff"] = df['timestamp'].diff()
df["Session Starters"] = (df["time_diff"] > pd.Timedelta(minutes=30)).cumsum()

/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_94788/3082888369.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'timestamp': pd.to_datetime([


In [76]:
'''Question 3: The "Poisoned" Preference (Data Quality)
Scenario: You decide to run DPO. You scrape "Chosen vs. Rejected" data from Reddit (upvotes vs. downvotes). However, your dataset is noisy.

Case A: The "Chosen" response is actually incorrect (hallucination), but it sounded confident so users upvoted it.

Case B: The "Rejected" response is factually true but was downvoted because it was rude.

If you train DPO on this, your model will learn to hallucinate confidently (Case A) or avoid facts (Case B).

The Question: You have a small budget to pay human labelers, but you can't re-label all 100k pairs. You can only verify 10% of the data.

How do you identify which 10% of the pairs are the "riskiest" or "most confusing" to the model?

What metric would you calculate using your current model to find these "high disagreement" pairs?
2. The "Model-Intrinsic" Metric (The Missing Piece) I asked for a metric using your current model (without paying for GPT-4 calls). The most efficient metric is the DPO Loss / Probability Margin.

Logic: You feed the "Chosen" and "Rejected" pair into your current model.

The Check: If your model assigns a higher probability to the "Rejected" answer than the "Chosen" answer, the Loss is high.

Conclusion: The model is "surprised" by the label.

Scenario A: The model is dumb and needs to learn.

Scenario B: The label is wrong (poisoned data).

Action: Sort all data by Probability(Rejected) - Probability(Chosen). The top 10% are the ones where your model and the dataset violently disagree. Fix those first.'''

'Question 3: The "Poisoned" Preference (Data Quality)\nScenario: You decide to run DPO. You scrape "Chosen vs. Rejected" data from Reddit (upvotes vs. downvotes). However, your dataset is noisy.\n\nCase A: The "Chosen" response is actually incorrect (hallucination), but it sounded confident so users upvoted it.\n\nCase B: The "Rejected" response is factually true but was downvoted because it was rude.\n\nIf you train DPO on this, your model will learn to hallucinate confidently (Case A) or avoid facts (Case B).\n\nThe Question: You have a small budget to pay human labelers, but you can\'t re-label all 100k pairs. You can only verify 10% of the data.\n\nHow do you identify which 10% of the pairs are the "riskiest" or "most confusing" to the model?\n\nWhat metric would you calculate using your current model to find these "high disagreement" pairs?\n2. The "Model-Intrinsic" Metric (The Missing Piece) I asked for a metric using your current model (without paying for GPT-4 calls). The most 

In [77]:
messages = [
    {"role": "system", "content": "You are a helpful assistant...", "tokens": 20},
    {"role": "user", "content": "Hi", "tokens": 5},
    {"role": "assistant", "content": "Hello!", "tokens": 5},
    # ... (many intermediate messages) ...
    {"role": "user", "content": "Summarize everything above.", "tokens": 10}
]

In [79]:
# keep the first system prompt 
# keep the last message 
# keep token count in between 
def prune_context(messages, max_tokens):
    system_prompt = messages[0]
    last_message = messages[-1]
    keep = []
    tokens = system_prompt["tokens"] + last_message["tokens"]
    # messages[1:-1]
    messages = messages[1:-1][::-1]
    for message in messages:
        tokens += message["tokens"]
        if tokens > max_tokens:
            break
        keep.append(message)
    return [system_prompt] + keep[::-1] + [last_message]

In [9]:
import pandas as pd

trades = pd.DataFrame({
    'time': pd.to_datetime(['09:00:00', '09:00:02', '09:00:05']),
    'sym': ['AAPL', 'MSFT', 'AAPL'],
    'qty': [10, 20, 10]
})

quotes = pd.DataFrame({
    'time': pd.to_datetime(['08:59:59', '09:00:01', '09:00:03', '09:00:06']),
    'sym': ['AAPL', 'AAPL', 'MSFT', 'AAPL'],
    'price': [150.0, 151.0, 300.0, 152.0]
})

/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_94788/2737861766.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'time': pd.to_datetime(['09:00:00', '09:00:02', '09:00:05']),
/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_94788/2737861766.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'time': pd.to_datetime(['08:59:59', '09:00:01', '09:00:03', '09:00:06']),


In [89]:
def merge_prices(trades, quotes):
    quotes = quotes.sort_values(by = "time")
    trades = trades.sort_values(by = "time")
    # Look for a specific pandas function designed for time-series alignment.
    return pd.merge_asof(
        trades, 
        quotes, 
        on="time",             # The timestamp to align
        by="sym",              # Match AAPL to AAPL (Exact match group)
        direction="backward",  # Look for the last price <= trade time
        suffixes=("", "_quote")
    )

merge_prices(trades, quotes)

,time,sym,qty,price
0,2025-12-13 09:00:00,AAPL,10,150.0
1,2025-12-13 09:00:02,MSFT,20,NaN
2,2025-12-13 09:00:05,AAPL,10,151.0


In [85]:
quotes

,time,sym,price
0,2025-12-13 08:59:59,AAPL,150.0
1,2025-12-13 09:00:01,AAPL,151.0
2,2025-12-13 09:00:03,MSFT,300.0
3,2025-12-13 09:00:06,AAPL,152.0


In [90]:
import pandas as pd

data = {
    'user_id': [1, 1, 1, 2, 2, 1],
    'timestamp': pd.to_datetime([
        '08:00', '08:15', '09:00', # User 1 (Gap > 30m between 08:15 and 09:00)
        '10:00', '10:05',          # User 2
        '09:10'                    # User 1 (Back again, 10 mins after 09:00)
    ])
}
df = pd.DataFrame(data).sort_values(by=['user_id', 'timestamp'])

/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_46219/2322102131.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'timestamp': pd.to_datetime([


In [92]:
df["session_id"] = (df["timestamp"].diff() > pd.Timedelta(minutes=45)).cumsum()

In [93]:
df

,user_id,timestamp,session_id
0,1,2025-12-14 08:00:00,0
1,1,2025-12-14 08:15:00,0
2,1,2025-12-14 09:00:00,0
5,1,2025-12-14 09:10:00,0
3,2,2025-12-14 10:00:00,1
4,2,2025-12-14 10:05:00,1


In [94]:
df["timestamp"].diff()


0               NaT
1   0 days 00:15:00
2   0 days 00:45:00
5   0 days 00:10:00
3   0 days 00:50:00
4   0 days 00:05:00
Name: timestamp, dtype: timedelta64[ns]

In [99]:
time_condition = df.groupby('user_id')['timestamp'].diff() > pd.Timedelta(minutes=45)
user_condition = df['user_id'] != df['user_id'].shift(1)
df['session_id'] = (time_condition | user_condition).cumsum()
df

,user_id,timestamp,session_id
0,1,2025-12-14 08:00:00,1
1,1,2025-12-14 08:15:00,1
2,1,2025-12-14 09:00:00,1
5,1,2025-12-14 09:10:00,1
3,2,2025-12-14 10:00:00,2
4,2,2025-12-14 10:05:00,2


In [101]:
user_condition

0     True
1    False
2    False
5    False
3     True
4    False
Name: user_id, dtype: bool

In [102]:
a = [1, 2, 3]
b = a
c = a[:]


In [103]:
a.append(3)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "user_id": [1, 2, 3, 4],
    "country": ["IN", "US", "IN", "FR"],
    "clicks": [10, 0, 5, 20],
    "purchases": [1, 0, 2, 4],
})

0    IN
1    US
2    IN
3    FR
Name: country, dtype: object

In [134]:

df["country"]
df["user_id"]
df["clicks"]
df[df["country"]=="IN"]
df[(df["clicks"]> 5) & (df["purchases"]>=2)]
df.iloc[0:2][["clicks", "user_id"]]


,clicks,user_id
0,10,1
1,0,2


In [135]:
df.loc[df["user_id"].isin([2, 4]), ["country", "purchases"]]


,country,purchases
1,US,0
3,FR,4


In [10]:
df.groupby("country", as_index=False).agg(total_clicks=("clicks", "sum"),total_purchases=("purchases", "sum"))

KeyError: 'country'

In [144]:
import numpy as np

a = np.array([[1, 2, 3],
              [4, 5, 6]])
b = np.array([10, 3, 5, 20, 7])

x = np.array([[1, 2, 3],
              [4, 5, 6]])
v = np.array([10, 20, 30])

In [150]:
a.shape
row_sums = a.sum(axis = 1, keepdims=True)
col_sums = a.sum(axis = 0, keepdims=True)

In [154]:
first_row = a[0, :] 

In [170]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "user_id": [1, 2, 3, 4, 5],
    "group": ["A", "A", "B", "B", "B"],
    "feat1": [10.0, 20.0, 15.0, 30.0, 25.0],
    "feat2": [100.0, 80.0, 120.0, 90.0, 110.0],
})
arr = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 0.0, 8.0],
    [6.0, 3.0, 0.0],
])

In [ ]:
df["feat1_norm"] = (df["feat1"] - df["feat1"].min()) / (df["feat1"].max() - df["feat1"].min())
df[["feat1", "feat2"]].apply(lambda x : (x - x.mean())/ x.std())
np.linalg.norm(arr, axis = 0 )
df["feat1_group_z"] = (
    df.groupby("group")["feat1"]
      .transform(lambda x: (x - x.mean()) / x.std())
)

users = users.merge(metrics, on="user_id", how = "left")
users users.fillna(0)
mask = users["score"]>=0.5
users[mask]

,user_id,country
0,1,IN
1,2,US
2,3,FR
3,4,IN


In [ ]:
users = pd.DataFrame({
    "user_id": [1, 2, 3, 4],
    "country": ["IN", "US", "FR", "IN"],
})

metrics = pd.DataFrame({
    "user_id": [1, 2, 4],
    "score": [0.9, 0.5, 0.75],
})

,user_id,country,score
0,1,IN,0.90
1,2,US,0.50
2,3,FR,0.00
3,4,IN,0.75


In [218]:
users

,user_id,country,score
0,1,IN,0.90
1,2,US,0.50
2,3,FR,NaN
3,4,IN,0.75


In [226]:
import pandas as pd
import numpy as np

events = pd.DataFrame({
    "user_id": [1, 1, 1, 2, 2, 3, 3, 3],
    "ts": [
        "2025-01-01 10:00:00",
        "2025-01-01 10:05:00",
        "2025-01-01 11:00:00",
        "2025-01-02 09:00:00",
        "2025-01-02 09:45:00",
        "2025-01-03 12:00:00",
        "2025-01-03 13:10:00",
        "2025-01-03 14:40:00",
    ],
    "event": [
        "view", "click", "purchase",
        "view", "view",
        "view", "click", "view"
    ],
    "amount": [0, 0, 100, 0, 0, 0, 0, 0]
})
events["ts"] = pd.to_datetime(events["ts"])
events = events.sort_values(["user_id", "ts"])

In [227]:
events

,user_id,ts,event,amount
0,1,2025-01-01 10:00:00,view,0
1,1,2025-01-01 10:05:00,click,0
2,1,2025-01-01 11:00:00,purchase,100
3,2,2025-01-02 09:00:00,view,0
4,2,2025-01-02 09:45:00,view,0
5,3,2025-01-03 12:00:00,view,0
6,3,2025-01-03 13:10:00,click,0
7,3,2025-01-03 14:40:00,view,0


In [248]:
# Define a session per user as:
# events where the time difference between consecutive events is ≤ 30 minutes;
# if gap > 30 minutes → new session.
events["time_diff"] = events["ts"].diff()
events["session_change"] = events["time_diff"]>pd.Timedelta(minutes=30)
events["session_id"] = events.groupby("user_id")["session_change"].cumsum()

In [266]:
events = events.sort_values(["user_id", "ts"])

# time diff per user
events["time_diff"] = events.groupby("user_id")["ts"].diff()

# mark session boundaries:
# - first event per user (NaT in time_diff)
# - or gap > 30 minutes
events["session_change"] = (
    events["time_diff"].isna() |
    (events["time_diff"] > pd.Timedelta(minutes=30))
)

# cumulative session id *within each user*
events["session_id"] = (
    events.groupby("user_id")["session_change"]
          .cumsum()
          .astype(int)
)
sessions = (
    events.groupby(["user_id", "session_id"])
          .agg(
              start_ts=("ts", "min"),
              end_ts=("ts", "max"),
              n_events=("event", "count"),
              total_amount=("amount", "sum"),
          )
          .reset_index()
)

In [282]:
import pandas as pd

orders = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "user_id":  [10, 10, 11, 11, 11, 12],
    "country":  ["IN", "IN", "US", "US", "US", "IN"],
    "amount":   [100, 150, 200, 50, 80, 120],
})


In [ ]:
subset_ = orders.groupby("user_id").agg(total_spent= ("amount", sum))
subset = subset_[subset_["total_spent"]>200]
total_amount = orders.groupby("country").agg(total_amount= ("amount", sum))

/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_46219/3268655868.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  subset_ = orders.groupby("user_id").agg(total_spent= ("amount", sum))


In [292]:
# using the same orders DataFrame, write code that gives, per country:
# num_users = number of unique users
# avg_amount = mean order amount

num_unique_users = orders.groupby("country")["user_id"].nunique()
avg_amount = orders.groupby("country")["amount"].mean()

In [293]:
orders.groupby("country").agg(num_unique_users = ("user_id", "nunique"), avg_amount = ("amount","mean") )

,num_unique_users,avg_amount
country,,
IN,2,123.333333
US,1,110.000000


In [303]:
orders = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "user_id":  [10, 10, 11, 11, 11, 12],
    "country":  ["IN", "IN", "US", "US", "US", "IN"],
    "amount":   [100, 150, 200, 50, 80, 120],
})
orders.groupby("country").agg(num_orders = ("order_id", "count"), median_amount = ("amount", "median"), max_order_amount= ("amount", "max") )

,num_orders,median_amount,max_order_amount
country,,,
IN,3,120.0,150
US,3,80.0,200


In [306]:
orders = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "user_id":  [10, 10, 11, 11, 11, 12],
    "country":  ["IN", "IN", "US", "US", "US", "IN"],
    "amount":   [100, 150, 200, 50, 80, 120],
})
orders

,order_id,user_id,country,amount
0,1,10,IN,100
1,2,10,IN,150
2,3,11,US,200
3,4,11,US,50
4,5,11,US,80
5,6,12,IN,120


In [314]:
orders["user_total_spent"] = orders.groupby("user_id")["amount"].transform("sum")
orders["high_value_user"] = orders["user_total_spent"] > 200

In [328]:
logfile = "log.txt"
records = []
with open(logfile, "r") as f: 
    for line in f : 
        parts = line.strip().split()
        print(parts)
        records.append({"timestamp": parts[0], 
                        "user_id": parts[1].replace("user_id=",""),
                        "event": parts[2].replace("event=",""),
                        "latency": int(parts[3].replace("latency_ms=", "")), 
                        "model": parts[4].replace("model=", "")})
df = pd.DataFrame(records, columns=["timestamp", "user_id", "event", "latency", "model"])
df.groupby("event").agg(avg_latency = ("latency", "mean"))

['2025-12-14T10:12:35Z', 'user_id=123', 'event=click', 'latency_ms=120', 'model=v1']
['2025-12-14T10:12:36Z', 'user_id=123', 'event=answer', 'latency_ms=450', 'model=v1']
['2025-12-14T10:13:01Z', 'user_id=456', 'event=click', 'latency_ms=80', 'model=v2']
['2025-12-14T10:13:03Z', 'user_id=456', 'event=error', 'latency_ms=300', 'model=v2']
['2025-12-14T10:13:10Z', 'user_id=123', 'event=click', 'latency_ms=100', 'model=v2']


,avg_latency
event,
answer,450.0
click,100.0
error,300.0


In [331]:
data = {
    "id": 1,
    "user": {
        "id": 123,
        "country": "IN",
        "profile": {
            "age": 30,
            "tier": "gold"
        }
    }
}
data

{'id': 1,
 'user': {'id': 123, 'country': 'IN', 'profile': {'age': 30, 'tier': 'gold'}}}

In [341]:
def flatten_user(data):
    for x, y  in data.items():
        if isinstance(data[x], dict):
            yield from flatten_user(data[x])
        else: 
            yield x, y 
dict(flatten_user(data))

{'id': 123, 'country': 'IN', 'age': 30, 'tier': 'gold'}

In [358]:
prompts = [
    "hi",
    "explain the theory of relativity in simple terms",
    "what is 2+2",
    "give me 50 ideas for a sci-fi novel about time travel and AI",
]
mask  = [x for x in prompts if len(x.split())>3]
print(mask)
mask.sort(key = lambda x:-len(x.split()))

['explain the theory of relativity in simple terms', 'give me 50 ideas for a sci-fi novel about time travel and AI']


In [360]:
logs = [
    {"user_id": 1, "event": "query", "text": "hello"},
    {"user_id": 2, "event": "query", "text": "help me"},
    {"user_id": 1, "event": "response", "text": "hi there"},
    {"user_id": 1, "event": "query", "text": "tell me a joke"},
]


In [ ]:
res = {}
for x in logs:
    if x["user_id"] not in res:
        res[x["user_id"]]= []
    res[x["user_id"]].append(x)
    

In [363]:
res

{1: [{'user_id': 1, 'event': 'query', 'text': 'hello'},
  {'user_id': 1, 'event': 'response', 'text': 'hi there'},
  {'user_id': 1, 'event': 'query', 'text': 'tell me a joke'}],
 2: [{'user_id': 2, 'event': 'query', 'text': 'help me'}]}

In [ ]:
def problematic_finetune(model, train_data, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1) # 1e-5 1e-6  
    # cosine decay scehduler 
    # warmup 
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for batch in train_data:
            outputs = model(batch['input_ids'])
            loss = criterion(outputs, batch['labels'])
            optimiser.zero_grad()
            loss.backward()
            optimizer.step()

    return model

In [ ]:
w = torch.ones(m,n)#Mx N  # pretrained 
r = # lora fientune 

W_new = w+ alpha * AB # mx r rx n 
# m x n 
# mr+rn m+n 
rank r= 8 / 64 
A = guassian 
B = zeros 



In [ ]:
# 7b param 
# 16 gb GPU
# adam  # momemtum , avg
# fp16 = weight, gradient
# full finetune 
fp16 = 2 bytes 
= weight(fp16) + weight(fp32) + 2 fp16 + gradients(fp32) 
= 2+ 4 + 2*2 + 4 
= 14 bytes per param 
14*

In [ ]:
activation memory - 


In [11]:
14*7

98

In [ ]:
98 gb of gpu memory | 16 gb GPU 
# LORA 